## **Advanced Analyitcs and Applications - Data Collection Strategy**

Desciption 

#### Team: 
- Robin Reiners
- Saied Farham Nia

##### **Table of Contents**

0. [Notebook Setup](#Notebook-Set-Up-and-Imports)
1. [Introduction](#Introduction)

7. [References](#References)

##### **Notebook Set Up and Imports**

In [1]:
%%html
<style>
.dataframe th {
    font-family: "JetBrainsMono Nerd Font";
}
.dataframe td {
    font-family: "JetBrainsMono Nerd Font";
}
</style>

In [2]:
import importlib
import os
import pickle
import subprocess
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import yaml

In [3]:
sys.path.append(str(Path.cwd().parent))
from src.utils.notebook_setup import load_files, setup_notebook

style_manager = setup_notebook()

if str(Path().resolve()).split("/")[-1] == "AAA":
    print("already set repo root")
else:
    notebooks_dir = Path().resolve()
    repo_root = notebooks_dir.parent
    config_dir = repo_root / "config"
    data_dir = repo_root / "data"
    results_dir = data_dir / "results"
    raw_data_dir = data_dir / "raw"
    processed_data_dir = data_dir / "processed"

    with open(config_dir / "config.yaml", "r") as file:
        config = yaml.safe_load(file)
    
    os.chdir(repo_root)

## Introduction
[Back to Table of Contents](#Table-of-Contents)

In [4]:
from src.api import taxi
importlib.reload(taxi)

<module 'src.api.taxi' from '/workspaces/AAA/src/api/taxi.py'>

In [8]:
from src.api.taxi import ChicagoTaxiAPI

api = ChicagoTaxiAPI(
    app_token=config["CHICAGO_TAXI"].get("APP_TOKEN", None)
)

df_sample = api.fetch_data(
    select="trip_id, company, fare, tips, payment_type, trip_start_timestamp",
    where="fare > 50 AND payment_type = 'Cash'",
    order="trip_start_timestamp DESC",
    limit=5
)


df_all_blue_ribbon = api.fetch_all_data(
    where="company = 'Blue Ribbon Taxi Association'",
    select="trip_id, fare, trip_miles, trip_start_timestamp",
    order="trip_start_timestamp DESC",
    records_to_fetch=150, # Fetch a total of 150 records
    batch_size=60,       # In batches of 60
    save_batches=True,     # Example of saving intermediate batches
    output_dir=Path("chicago_taxi_blue_ribbon_batches")
)

        
socrata_types = api.get_column_socrata_types()

for i, (col, dtype) in enumerate(socrata_types.items()):
    if i < 10: # Print first 10
        print(f"  {col}: {dtype}")
    elif i == 10:
        print("  ... and more.")
        break
    else:
        print("Could not retrieve Socrata column types.")



api.close()

Fetching all data: 100%|██████████| 150/150 [00:00<00:00, 184.31rec/s]

🔗 Combining all fetched batches...
⚙️ Applying type conversions to combined data...
✅ Total data fetched: 150 records.
  trip_id: text
  taxi_id: text
  trip_start_timestamp: calendar_date
  trip_end_timestamp: calendar_date
  trip_seconds: number
  trip_miles: number
  pickup_census_tract: text
  dropoff_census_tract: text
  pickup_community_area: number
  dropoff_community_area: number
  ... and more.


In [ ]:
df_sample

In [25]:
from src.api.taxi import ChicagoTaxiAPI

taxi_api = ChicagoTaxiAPI()

# Get metadata
metadata = taxi_api.get_metadata()
if metadata:
    name = metadata.get("name")
    rows = int(metadata.get("columns")[0]["cachedContents"]["count"])

    print(f"📊 Dataset:\t{name}")
    # print formatted number with seperators 
    print(f"📊 Rows:\t{rows:,}")

📊 Dataset:	Taxi Trips (2024-)
📊 Rows:	8,495,661


In [27]:
api = ChicagoTaxiAPI()

where = api.build_where_clause(
    pickup_centroid_location__isnotnull=True,
    dropoff_centroid_location__isnotnull=True,
    trip_start_timestamp__gte="trip_end_timestamp",
    trip_start_timestamp__isnotnull=True,
)

api.fetch_data(limit=10, where_clause=where, order_by="trip_start_timestamp ASC, pickup_community_area ASC")

❌ API request failed: 400 Client Error: Bad Request for url: https://data.cityofchicago.org/resource/ajtu-isnz.json?%24limit=10&%24offset=0&%24order=trip_start_timestamp+ASC%2C+pickup_community_area+ASC&%24where=pickup_centroid_location+IS+NOT+NULL+AND+dropoff_centroid_location+IS+NOT+NULL+AND+trip_start_timestamp+%3E%3D+%27trip_end_timestamp%27+AND+trip_start_timestamp+IS+NOT+NULL


In [28]:
where

"pickup_centroid_location IS NOT NULL AND dropoff_centroid_location IS NOT NULL AND trip_start_timestamp >= 'trip_end_timestamp' AND trip_start_timestamp IS NOT NULL"

In [21]:
taxi_api.fetch_in_batches(
    total_records=rows,
    batch_size=50_000,
)

Fetching records:  64%|██████▍   | 5450000/8495661 [19:41<11:00, 4614.19rows/s]

❌ API request failed: HTTPSConnectionPool(host='data.cityofchicago.org', port=443): Read timed out. (read timeout=60)
⚠️  No more data available at offset 5450000
🔗 Combining batches...


✅ Saved combined dataset: 4,922,401 records to data/raw/taxi_data_combined.parquet 
🔺 Filtered out 0 records 


,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,...,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,pickup_census_tract,dropoff_census_tract
0,0a11567bd996c51ac32efbb3e60f923c063aabdc,55ef60184d508809c50a84b8c378a8572fcc0f5bd2ff49...,2025-05-01 00:00:00,2025-05-01 00:00:00,291.0,1.27,3.0,6.0,6.25,0.00,...,0.0,6.25,Prcard,Flash Cab,41.965812,-87.655879,41.944227,-87.655998,<NA>,<NA>
1,122e816a1b04575c94de158252e4e3a544b9f900,52c3ffa685a3b5ced3d16461deec5a0326086bee3d8f6a...,2025-05-01 00:00:00,2025-05-01 00:15:00,1733.0,14.95,76.0,3.0,38.75,8.65,...,4.0,51.90,Credit Card,Sun Taxi,41.980264,-87.913625,41.965812,-87.655879,<NA>,<NA>
2,1d17183a28c299443a40163dc3e6d50e10f054f1,d511072131b602026bdb9faa5491d15c3af8d62dc00659...,2025-05-01 00:00:00,2025-05-01 00:00:00,420.0,3.10,8.0,32.0,10.75,0.00,...,0.0,10.75,Cash,Taxi Affiliation Services,41.899602,-87.633308,41.878866,-87.625192,<NA>,<NA>
3,25264cb08e7db926efd88cab3c35d1df4995e11c,c84c28526a906ef1ad0ea7dc570f97949ecf92dfe156cb...,2025-05-01 00:00:00,2025-05-01 00:15:00,882.0,7.72,76.0,17.0,21.00,0.00,...,6.0,27.00,Cash,Globe Taxi,41.980264,-87.913625,41.946511,-87.806020,<NA>,<NA>
4,3592419fe47b3f8b8c6e9df61d50421dd71a32e1,f32c0c8e63d4e5d4bfc7e9d57e5d0f6dcf28450850245d...,2025-05-01 00:00:00,2025-05-01 00:15:00,1560.0,12.70,76.0,2.0,33.25,7.55,...,4.0,44.80,Credit Card,Taxi Affiliation Services,41.980264,-87.913625,42.001571,-87.695013,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4922396,0966f26090fdd833cd1307f363f74f71d58f4406,7ff7185bd7ad803670224d5d809fe1160145d1d647ad7d...,2024-06-24 21:15:00,2024-06-24 21:15:00,243.0,0.86,28.0,24.0,5.50,1.00,...,0.0,7.00,Credit Card,Sun Taxi,41.874005,-87.663518,41.901207,-87.676356,<NA>,<NA>
4922397,0ffc6273221d44642cef644745f21d454d7544cc,52ca9a97ddafa021d0588c76040babc4e8ddeb917c9dc4...,2024-06-24 21:15:00,2024-06-24 21:30:00,982.0,6.68,8.0,3.0,19.00,0.00,...,0.0,19.00,Cash,Flash Cab,41.899602,-87.633308,41.965812,-87.655879,<NA>,<NA>
4922398,1041c1166e509acce6da91a19207460875b3c18e,56a1119c6ca57e39525cf06829f9ecff553cf4b5ac2482...,2024-06-24 21:15:00,2024-06-24 21:30:00,1425.0,8.58,8.0,14.0,22.41,5.44,...,0.0,27.85,Mobile,Medallion Leasin,41.899602,-87.633308,41.968069,-87.721559,<NA>,<NA>
4922399,12a27f8abd6ea81db927c05aa71f58d8a5294d69,4ab868a2a19b95f50f355eb8d0d7638f1912b37f49ebd6...,2024-06-24 21:15:00,2024-06-24 21:45:00,2153.0,10.22,32.0,2.0,29.50,6.00,...,0.0,35.50,Prcard,Flash Cab,41.878866,-87.625192,42.001571,-87.695013,<NA>,<NA>


In [23]:
taxi_api.fetch_data(limit=10, offset=5450000)

❌ API request failed: HTTPSConnectionPool(host='data.cityofchicago.org', port=443): Read timed out. (read timeout=60)
